In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import PROJECT.util as f
from matplotlib.pyplot import cm
from mpl_toolkits import mplot3d
import plotly.graph_objects as go
import pandas as pd
import pickle
from plotly.subplots import make_subplots
import bz2

In [ ]:
def overlap(x,x_,d):
    return np.abs(np.einsum('i,ji->j', x, x_)/d)

def mse1(x_e, array_x):
    ## x-y
    mse=((np.abs(x_e)-np.abs(array_x))**2).mean(axis=1)
    return mse
def mse2(u_e, v_e, array_u, array_v, Y):
    ## u*v*-uv
    mse = np.tensordot(u_e, v_e, 0) - np.einsum('ad,ah->adh',array_u, array_v)
    return (mse**2).mean(axis=(1,2))
def mse3(u_t, v_t, Y):
    ## Y-uv
    tmp = Y - np.einsum('ad,ah->adh',array_u, array_v)
    return (mse**2).mean(axis=(1,2))

In [ ]:
# WHERE TO READ
FILE_PATH="C:/Users/Admin/Desktop/"

# enter the configuration
name_n_1="200"
name_m_1="200"
lambda_u_val_1="1"
lambda_v_val_1="1"
lambda_val_1="2"

name_value_1="beta_u"
name_value_2="beta_v"

filename =f"n={name_n_1}_m={name_m_1}_lu={lambda_u_val_1}_lv={lambda_v_val_1}_l={lambda_val_1}.pickle"

infile = bz2.BZ2File(FILE_PATH+filename,'rb')
(main_res_u_new, main_res_v_new, main_config_new, save_checking_range_new) = pickle.load(infile)
infile.close()

In [ ]:
list_value_1=save_checking_range_new[0]
list_value_2=save_checking_range_new[1]

# WHERE TO WRITE THE PLOTS
FILE_PATH_W="C:/Users/Admin/Desktop/graph/"
# Pick a mse
name_mse='mse1'

In [ ]:
## COMPUTE THE CHOOSEN MSE:
mse_at_each_iteration=[]
mse_at_each_iteration_u=[]
mse_at_each_iteration_v=[]
if(name_mse=='mse1'):
    mse_at_each_iteration_u=[]
    mse_at_each_iteration_v=[]
    for i in range(len(main_config_new)):
        # intermediate array to split the main array:
        # shape(nb_run*sample_size) to shape((nb_run, sample_size))
        one_sample_u=[]
        one_sample_v=[]
        for j in range(len(main_config_new[0])):
            one_sample_u.append(mse1(main_config_new[i][j][0]['u_'],
                                      main_res_u_new[i][j]))
            one_sample_v.append(mse1(main_config_new[i][j][0]['v_'],
                                      main_res_v_new[i][j]))
        mse_at_each_iteration_u.append(one_sample_u)
        mse_at_each_iteration_v.append(one_sample_v)
elif(name_mse=='mse2'):
    mse_at_each_iteration=[]
    for i in range(len(main_config_new)):
        # intermediate array to split the main array:
        # shape(nb_run*sample_size) to shape((nb_run, sample_size))
        one_sample=[]
        for j in range(len(main_config_new[0])):
            one_sample.append(mse2(main_config_new[i][j][0]['u_'],
                                      main_config_new[i][j][0]['v_'],
                                      main_res_u_new[i][j],
                                      main_res_v_new[i][j],
                                      main_config_new[i][j][0]['Y']))
        mse_at_each_iteration.append(one_sample)
elif(name_mse=='mse3'):
    mse_at_each_iteration=[]
    for i in range(len(main_config_new)):
        # intermediate array to split the main array:
        # shape(nb_run*sample_size) to shape((nb_run, sample_size))
        one_sample=[]
        for j in range(len(main_config_new[0])):
            one_sample.append(mse3(main_res_u_new[i][j],
                                   main_res_v_new[i][j],
                                   main_config_new[i][j][0]['Y']))
        mse_at_each_iteration.append(one_sample)

In [ ]:
mse_at_each_iteration=np.array(mse_at_each_iteration)
mse_at_each_iteration_u=np.array(mse_at_each_iteration_u)
mse_at_each_iteration_v=np.array(mse_at_each_iteration_v)

if(name_mse=='mse1'):
    # MSE U
    mse_at_final_iteration_u = mse_at_each_iteration_u[:,:,-1].mean(axis=1)

    # MSE V
    mse_at_final_iteration_v = mse_at_each_iteration_v[:,:,-1].mean(axis=1)
    
    # BOTH
    mse_at_each_iteration = np.array([mse_at_each_iteration_u, mse_at_each_iteration_v]).mean(0)
elif(name_mse=='mse2' or name_mse=='mse3'):
    # MSE U and V are the same in this configuration:
    mse_at_each_iteration_u = mse_at_each_iteration
    mse_at_final_iteration_u = mse_at_each_iteration_u[:,:,-1].mean(axis=1)
    mse_at_each_iteration_v = mse_at_each_iteration
    mse_at_final_iteration_v = mse_at_each_iteration_v[:,:,-1].mean(axis=1)

# Take last iteration and mean between sample:
mse_at_final_iteration = mse_at_each_iteration[:,:,-1].mean(axis=1)

In [ ]:
## COMPUTE OVERLAP AT EACH ITERATION

# OVERLAP FOR U:
overlap_at_each_iteration_u=[]
for i in range(len(main_config_new)):
    # intermediate array to split the main array:
    # shape(nb_run*sample_size) to shape((nb_run, sample_size))
    one_sample=[]
    for j in range(len(main_config_new[0])):
        one_sample.append(overlap(main_config_new[i][j][0]['u_'],
                                  main_res_u_new[i][j],
                                  main_config_new[i][j][0]['N']))
    overlap_at_each_iteration_u.append(one_sample)
    
# OVERLAP FOR U:
overlap_at_each_iteration_v=[]
for i in range(len(main_config_new)):
    # intermediate array to split the main array:
    # shape(nb_run*sample_size) to shape((nb_run, sample_size))
    one_sample=[]
    for j in range(len(main_config_new[0])):
        one_sample.append(overlap(main_config_new[i][j][0]['v_'],
                                  main_res_v_new[i][j],
                                  main_config_new[i][j][0]['N']))
    overlap_at_each_iteration_v.append(one_sample)

In [ ]:
# OVERLAP U
overlap_at_each_iteration_u = np.array(overlap_at_each_iteration_u)
overlap_at_final_iteration_u = overlap_at_each_iteration_u[:,:,-1].mean(axis=1)

# OVERLAP V
overlap_at_each_iteration_v = np.array(overlap_at_each_iteration_v)
overlap_at_final_iteration_v = overlap_at_each_iteration_v[:,:,-1].mean(axis=1)

# BOTH
overlap_at_each_iteration = np.mean([overlap_at_each_iteration_u, overlap_at_each_iteration_v], axis=0)
# Take last iteration and mean between sample:
overlap_at_final_iteration = overlap_at_each_iteration[:,:,-1].mean(axis=1)

mse_at_final_iteration=mse_at_final_iteration.reshape(len(list_value_1),len(list_value_2))
overlap_at_final_iteration=overlap_at_final_iteration.reshape(len(list_value_1),len(list_value_2))

In [ ]:
# OVERLAP AND MSE
fig = make_subplots(rows=1, cols=2, specs=[[{'type': 'surface'}, {'type': 'surface'}]])
fig.add_trace(go.Surface(z=overlap_at_final_iteration, x=list_value_2, y=list_value_1,
                         name="overlap", showscale=False),
             row=1, col=1)
fig.add_trace(go.Surface(z=mse_at_final_iteration, x=list_value_2, y=list_value_1,
                         name="mse", showscale=False, colorscale ='viridis'),
             row=1, col=2)

fig.update_layout(scene = dict(
                    xaxis_title=name_value_2,
                    yaxis_title=name_value_1,
                    zaxis_title="overlap"),
                  scene2 = dict(
                    xaxis_title=name_value_2,
                    yaxis_title=name_value_1,
                    zaxis_title="mse"),
               autosize=False, width=1000, height=500)

#title_text=f"N={name_n_1}, M={name_m_1}, lu={lambda_u_val_1}, lv={lambda_v_val_1}, l={lambda_val_1}")

fig.update_traces(showlegend=True)

fig.show()
path=f"{FILE_PATH_W}n={name_n_1}_m={name_m_1}_lu={lambda_u_val_1}_lv={lambda_v_val_1}_l={lambda_val_1}.html"
fig.write_html(path)

In [ ]:
## HEAT MAP VARIANCE
variance_map = np.var(overlap_at_each_iteration[:,:,-1], axis=1)
variance_map = variance_map.reshape(len(list_value_1),len(list_value_2))

variance_mse_map = np.var(mse_at_each_iteration[:,:,-1], axis=1)
variance_mse_map = variance_mse_map.reshape(len(list_value_1),len(list_value_2))

x, y = np.meshgrid(list_value_2,list_value_1)

fig,ax=plt.subplots(1,2, figsize=(12,5), facecolor='white')
#fig.tight_layout()
fig.subplots_adjust(top=0.8)
fig.suptitle(f"n={name_n_1}, m={name_m_1}; "+
             f"lambda_u={lambda_u_val_1}, lambda_v={lambda_v_val_1}; "+
             f"lambda={lambda_val_1}", size=16)

bar0=ax[0].contourf(x,y, variance_map)
ax[0].set_title("VARIANCE OVERLAP")
ax[0].set_xlabel(f"{name_value_2}")
ax[0].set_ylabel(f"{name_value_1}")

bar1 = ax[1].contourf(x,y, variance_mse_map)
ax[1].set_title("VARIANCE MSE")
ax[1].set_xlabel(f"{name_value_2}")
ax[1].set_ylabel(f"{name_value_1}")

plt.colorbar(bar0, ax=ax[0])
plt.colorbar(bar1, ax=ax[1])

path=f"{FILE_PATH_W}n={name_n_1}_m={name_m_1}_lu={lambda_u_val_1}_lv={lambda_v_val_1}_l={lambda_val_1}_variance.jpg"
plt.savefig(path)

plt.show()

In [ ]:
# OVERLAP AND MSE RUN 1 _ U et V séparés!

overlap_at_final_iteration_u=overlap_at_final_iteration_u.reshape(len(list_value_1),len(list_value_2))
overlap_at_final_iteration_v=overlap_at_final_iteration_v.reshape(len(list_value_1),len(list_value_2))
#overlap_at_final_iteration_v=overlap_at_final_iteration_v+0.1

mse_at_final_iteration_u=mse_at_final_iteration_u.reshape(len(list_value_1),len(list_value_2))
mse_at_final_iteration_v=mse_at_final_iteration_v.reshape(len(list_value_1),len(list_value_2))
#mse_at_final_iteration_v=mse_at_final_iteration_v+0.1

fig = make_subplots(rows=1, cols=2, specs=[[{'type': 'surface'}, {'type': 'surface'}]])
fig.add_trace(go.Surface(z=overlap_at_final_iteration_u, x=list_value_2, y=list_value_1,
                         name="overlap_u", showscale=False),
             row=1, col=1)
fig.add_trace(go.Surface(z=overlap_at_final_iteration_v, x=list_value_2, y=list_value_1,
                         name="overlap_v", showscale=False, opacity=0.9),
             row=1, col=1)

fig.add_trace(go.Surface(z=mse_at_final_iteration_u, x=list_value_2, y=list_value_1,
                         name="mse_u", showscale=False, colorscale ='viridis'),
             row=1, col=2)
fig.add_trace(go.Surface(z=mse_at_final_iteration_v, x=list_value_2, y=list_value_1,
                         name="mse_v", showscale=False, colorscale ='viridis', opacity=0.9),
             row=1, col=2)

fig.update_layout(
    title=f"n={name_n_1}, m={name_m_1}; lu={lambda_u_val_1}, lv={lambda_v_val_1}, l={lambda_val_1}"
)
fig.update_layout(scene = dict(
                    xaxis_title=name_value_2,
                    yaxis_title=name_value_1,
                    zaxis_title="overlap"),
                  scene2 = dict(
                    xaxis_title=name_value_2,
                    yaxis_title=name_value_1,
                    zaxis_title="mse"),
               autosize=False, width=1000, height=500)
fig.update_traces(showlegend=True)

fig.show()
path=f"{FILE_PATH_W}n={name_n_1}_m={name_m_1}_lu={lambda_u_val_1}_lv={lambda_v_val_1}_l={lambda_val_1}_uv.html"
fig.write_html(path)